# Summarizing and Computing Descriptive Statistics

pandas objects are equipped with a set of common mathematical and statistical methods.
Most of these fall into the category of reductions or summary statistics, methods
that extract a single value (like the sum or mean) from a Series or a Series of values from
the rows or columns of a DataFrame. Compared with the equivalent methods of vanilla
NumPy arrays, they are all built from the ground up to exclude missing data. Consider a small DataFrame:

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [8]:
df = DataFrame([[1.4, np.nan], [7.1, -4.5], 
                [np.nan, np.nan], [0.75, -1.3]],
               index=['a', 'b', 'c', 'd'],columns=['one', 'two'])

In [9]:
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


Calling DataFrame’s sum method returns a Series containing column sums:

In [10]:
df.sum()

one    9.25
two   -5.80
dtype: float64

Passing axis=1 sums over the rows instead:

In [11]:
df.sum(axis=1)

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

NA values are excluded unless the entire slice (row or column in this case) is NA. This
can be disabled using the skipna option:

In [13]:
df.mean(axis=1, skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

See Table 5-9 for a list of common options for each reduction method options.

Table 5-9. Options for reduction methods

Method Description

axis Axis to reduce over. 0 for DataFrame’s rows and 1 for columns.

skipna Exclude missing values, True by default.

level Reduce grouped by level if the axis is hierarchically-indexed (MultiIndex).

Some methods, like idxmin and idxmax, return indirect statistics like the index value
where the minimum or maximum values are attained:

In [15]:
df.idxmax()

one    b
two    d
dtype: object

Other methods are accumulations:

In [16]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


Another type of method is neither a reduction nor an accumulation. describe is one
such example, producing multiple summary statistics in one shot:

In [17]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


On non-numeric data, describe produces alternate summary statistics:

In [19]:
obj = Series(['a', 'b', 'c', 'd'] * 4 )

In [20]:
obj.describe()

count     16
unique     4
top        d
freq       4
dtype: object

See Table 5-10 for a full list of summary statistics and related methods.

Table 5-10. Descriptive and summary statistics

Method Description

count Number of non-NA values

describe Compute set of summary statistics for Series or each DataFrame column

min, max Compute minimum and maximum values

argmin, argmax Compute index locations (integers) at which minimum or maximum value obtained, respectively

idxmin, idxmax Compute index values at which minimum or maximum value obtained, respectively

quantile Compute sample quantile ranging from 0 to 1

sum Sum of values

mean Mean of values

median Arithmetic median (50% quantile) of values

mad Mean absolute deviation from mean value

var Sample variance of values

std Sample standard deviation of values

skew Sample skewness (3rd moment) of values

kurt Sample kurtosis (4th moment) of values

cumsum Cumulative sum of values

cummin, cummax Cumulative minimum or maximum of values, respectively

cumprod Cumulative product of values

diff Compute 1st arithmetic difference (useful for time series)

pct_change Compute percent changes

## Correlation and Covariance

Some summary statistics, like correlation and covariance, are computed from pairs of
arguments. Let’s consider some DataFrames of stock prices and volumes obtained from
Yahoo! Finance:

In [24]:
#import pandas.io.data as web
import pandas_datareader.data as web


# 183
# down vote
# accepted
# As you are in python3 , use dict.items() instead of dict.iteritems()
# iteritems() was removed in python3, so you can't use this method anymore.
# Take a look at Python Wiki (Link)

all_data = {}
for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']:
    all_data[ticker] = web.get_data_yahoo(ticker, '1/1/2000', '1/1/2010')

price = DataFrame({tic: data['Adj Close']
    for tic, data in all_data.items()})

volume = DataFrame({tic: data['Volume']
    for tic, data in all_data.items()})

I now compute percent changes of the prices:

In [26]:
returns = price.pct_change()

In [27]:
returns.tail()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2009-12-24,0.034339,0.011117,0.004385,0.002587
2009-12-28,0.012294,0.007098,0.013326,0.005484
2009-12-29,-0.011861,-0.005571,-0.003477,0.007058
2009-12-30,0.012147,0.005376,0.005461,-0.013699
2009-12-31,-0.004300,-0.004416,-0.012597,-0.015504


The corr method of Series computes the correlation of the overlapping, non-NA,
aligned-by-index values in two Series. Relatedly, cov computes the covariance:

In [30]:
returns.MSFT.corr(returns.IBM)

0.49597963862836758

In [31]:
returns.MSFT.cov(returns.IBM)

0.00021595761148660868

DataFrame’s corr and cov methods, on the other hand, return a full correlation or
covariance matrix as a DataFrame, respectively:m

In [33]:
returns.corr()

,AAPL,GOOG,IBM,MSFT
AAPL,1.000000,0.470676,0.410011,0.424305
GOOG,0.470676,1.000000,0.390689,0.443587
IBM,0.410011,0.390689,1.000000,0.495980
MSFT,0.424305,0.443587,0.495980,1.000000


In [34]:
returns.cov()

,AAPL,GOOG,IBM,MSFT
AAPL,0.001027,0.000303,0.000252,0.000309
GOOG,0.000303,0.000580,0.000142,0.000205
IBM,0.000252,0.000142,0.000367,0.000216
MSFT,0.000309,0.000205,0.000216,0.000516


Using DataFrame’s corrwith method, you can compute pairwise correlations between
a DataFrame’s columns or rows with another Series or DataFrame. Passing a Series
returns a Series with the correlation value computed for each column:

In [35]:
returns.corrwith(returns.IBM)

AAPL    0.410011
GOOG    0.390689
IBM     1.000000
MSFT    0.495980
dtype: float64

Passing a DataFrame computes the correlations of matching column names. Here I
compute correlations of percent changes with volume:

In [36]:
returns.corrwith(volume)

AAPL   -0.057549
GOOG    0.062647
IBM    -0.007892
MSFT   -0.014245
dtype: float64

Passing axis=1 does things row-wise instead. In all cases, the data points are aligned by
label before computing the correlation.

## Unique Values, Value Counts, and Membership

Another class of related methods extracts information about the values contained in a
one-dimensional Series. To illustrate these, consider this example:


In [37]:
obj = Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

The first function is unique, which gives you an array of the unique values in a Series:

In [38]:
uniques = obj.unique()

In [39]:
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

The unique values are not necessarily returned in sorted order, but could be sorted after
the fact if needed (uniques.sort()). Relatedly, value_counts computes a Series containing
value frequencies:

In [40]:
obj.value_counts()

a    3
c    3
b    2
d    1
dtype: int64

The Series is sorted by value in descending order as a convenience. value_counts is also
available as a top-level pandas method that can be used with any array or sequence:m

In [43]:
pd.value_counts(obj.values, sort=False)

c    3
a    3
b    2
d    1
dtype: int64

Lastly, isin is responsible for vectorized set membership and can be very useful in
filtering a data set down to a subset of values in a Series or column in a DataFrame:

In [44]:
mask = obj.isin(['b', 'c'])

In [45]:
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [46]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

Table 5-11. Unique, value counts, and binning methods

Method Description

isin Compute boolean array indicating whether each Series value is contained in the passed sequence of values.

unique Compute array of unique values in a Series, returned in the order observed.

value_counts Return a Series containing unique values as its index and frequencies as its values, ordered count in
descending order.

In some cases, you may want to compute a histogram on multiple related columns in
a DataFrame. Here’s an example:

In [47]:
data = DataFrame({'Qu1': [1, 3, 4, 3, 4],
        'Qu2': [2, 3, 1, 2, 3],
        'Qu3': [1, 5, 2, 4, 4]})

In [48]:
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


Passing pandas.value_counts to this DataFrame’s apply function gives:

In [49]:
result = data.apply(pd.value_counts).fillna(0)

In [50]:
result

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0
